[View in Colaboratory](https://colab.research.google.com/github/mxu007/deeplearning.ai/blob/master/tensorflow/notebooks/5-conv-draft.ipynb)

# Introduction to Convolutional Neural Networks (convnets or CNNs) with tf.keras, tf.data, and eager execution.

For this tutorial we will look at a convnet problem. Like the first tutorial, we will classify MNIST digits. This tutorial was inspired by [this one](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.1-introduction-to-convnets.ipynb).

In [1]:
!pip install -q -U tensorflow==1.8.0
import tensorflow as tf

import numpy as np

# Enable Eager mode. Re-running this cell will fail. Restart the Runtime to re-enable Eager.
tf.enable_eager_execution()

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

train_images = np.asarray(train_images, dtype=np.float32) / 255

# Convert the train images and add channels
train_images = train_images.reshape((TRAINING_SIZE, 28, 28, 1))

test_images = np.asarray(test_images, dtype=np.float32) / 255
# Convert the train images and add channels
test_images = test_images.reshape((TEST_SIZE, 28, 28, 1))

In [4]:
# How many digits we are predicting from (0-9)
LABEL_DIMENSIONS = 10

train_labels  = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels, LABEL_DIMENSIONS)

# Cast the labels to floats, needed later
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [5]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu, input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

In [7]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [8]:
BATCH_SIZE=128

# Because tf.data may work with potentially **large** collections of data
# we do not shuffle the entire dataset by default
# Instead, we maintain a buffer of SHUFFLE_SIZE elements
# and sample from there.
SHUFFLE_SIZE = 10000 

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.shuffle(SHUFFLE_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [9]:
EPOCHS=5

for epoch in range(EPOCHS):
  for (batch, (images, labels)) in enumerate(dataset):
    train_loss, train_accuracy = model.train_on_batch(images, labels)
    
    if batch % 10 == 0: print(batch, train_accuracy)
  
  # Here you can gather any metrics or adjust your training parameters
  print('Epoch #%d\t Loss: %.6f\tAccuracy: %.6f' % (epoch + 1, train_loss, train_accuracy))

(0, <tf.Tensor: id=513, shape=(), dtype=float32, numpy=0.1328125>)
(10, <tf.Tensor: id=2214, shape=(), dtype=float32, numpy=0.1015625>)
(20, <tf.Tensor: id=3915, shape=(), dtype=float32, numpy=0.1015625>)
(30, <tf.Tensor: id=5616, shape=(), dtype=float32, numpy=0.140625>)
(40, <tf.Tensor: id=7317, shape=(), dtype=float32, numpy=0.1171875>)
(50, <tf.Tensor: id=9018, shape=(), dtype=float32, numpy=0.1796875>)
(60, <tf.Tensor: id=10719, shape=(), dtype=float32, numpy=0.328125>)
(70, <tf.Tensor: id=12420, shape=(), dtype=float32, numpy=0.3984375>)
(80, <tf.Tensor: id=14121, shape=(), dtype=float32, numpy=0.5234375>)
(90, <tf.Tensor: id=15822, shape=(), dtype=float32, numpy=0.484375>)
(100, <tf.Tensor: id=17523, shape=(), dtype=float32, numpy=0.6171875>)
(110, <tf.Tensor: id=19224, shape=(), dtype=float32, numpy=0.6796875>)
(120, <tf.Tensor: id=20925, shape=(), dtype=float32, numpy=0.796875>)
(130, <tf.Tensor: id=22626, shape=(), dtype=float32, numpy=0.8203125>)
(140, <tf.Tensor: id=24327, 

(50, <tf.Tensor: id=88802, shape=(), dtype=float32, numpy=0.9921875>)
(60, <tf.Tensor: id=90503, shape=(), dtype=float32, numpy=0.9765625>)
(70, <tf.Tensor: id=92204, shape=(), dtype=float32, numpy=0.96875>)
(80, <tf.Tensor: id=93905, shape=(), dtype=float32, numpy=0.96875>)
(90, <tf.Tensor: id=95606, shape=(), dtype=float32, numpy=0.9921875>)
(100, <tf.Tensor: id=97307, shape=(), dtype=float32, numpy=0.9921875>)
(110, <tf.Tensor: id=99008, shape=(), dtype=float32, numpy=0.9765625>)
(120, <tf.Tensor: id=100709, shape=(), dtype=float32, numpy=0.984375>)
(130, <tf.Tensor: id=102410, shape=(), dtype=float32, numpy=0.96875>)
(140, <tf.Tensor: id=104111, shape=(), dtype=float32, numpy=0.9921875>)
(150, <tf.Tensor: id=105812, shape=(), dtype=float32, numpy=0.953125>)
(160, <tf.Tensor: id=107513, shape=(), dtype=float32, numpy=1.0>)
(170, <tf.Tensor: id=109214, shape=(), dtype=float32, numpy=0.984375>)
(180, <tf.Tensor: id=110915, shape=(), dtype=float32, numpy=0.9921875>)
(190, <tf.Tensor: i

(90, <tf.Tensor: id=175389, shape=(), dtype=float32, numpy=0.9765625>)
(100, <tf.Tensor: id=177090, shape=(), dtype=float32, numpy=0.984375>)
(110, <tf.Tensor: id=178791, shape=(), dtype=float32, numpy=1.0>)
(120, <tf.Tensor: id=180492, shape=(), dtype=float32, numpy=0.984375>)
(130, <tf.Tensor: id=182193, shape=(), dtype=float32, numpy=0.9921875>)
(140, <tf.Tensor: id=183894, shape=(), dtype=float32, numpy=0.984375>)
(150, <tf.Tensor: id=185595, shape=(), dtype=float32, numpy=0.984375>)
(160, <tf.Tensor: id=187296, shape=(), dtype=float32, numpy=0.984375>)
(170, <tf.Tensor: id=188997, shape=(), dtype=float32, numpy=1.0>)
(180, <tf.Tensor: id=190698, shape=(), dtype=float32, numpy=0.9921875>)
(190, <tf.Tensor: id=192399, shape=(), dtype=float32, numpy=0.9921875>)
(200, <tf.Tensor: id=194100, shape=(), dtype=float32, numpy=0.96875>)
(210, <tf.Tensor: id=195801, shape=(), dtype=float32, numpy=0.9921875>)
(220, <tf.Tensor: id=197502, shape=(), dtype=float32, numpy=0.9921875>)
(230, <tf.Te

(130, <tf.Tensor: id=261976, shape=(), dtype=float32, numpy=0.9921875>)
(140, <tf.Tensor: id=263677, shape=(), dtype=float32, numpy=0.984375>)
(150, <tf.Tensor: id=265378, shape=(), dtype=float32, numpy=0.984375>)
(160, <tf.Tensor: id=267079, shape=(), dtype=float32, numpy=0.984375>)
(170, <tf.Tensor: id=268780, shape=(), dtype=float32, numpy=1.0>)
(180, <tf.Tensor: id=270481, shape=(), dtype=float32, numpy=1.0>)
(190, <tf.Tensor: id=272182, shape=(), dtype=float32, numpy=0.984375>)
(200, <tf.Tensor: id=273883, shape=(), dtype=float32, numpy=1.0>)
(210, <tf.Tensor: id=275584, shape=(), dtype=float32, numpy=1.0>)
(220, <tf.Tensor: id=277285, shape=(), dtype=float32, numpy=0.9921875>)
(230, <tf.Tensor: id=278986, shape=(), dtype=float32, numpy=0.9765625>)
(240, <tf.Tensor: id=280687, shape=(), dtype=float32, numpy=0.9921875>)
(250, <tf.Tensor: id=282388, shape=(), dtype=float32, numpy=0.984375>)
(260, <tf.Tensor: id=284089, shape=(), dtype=float32, numpy=0.984375>)
(270, <tf.Tensor: id=2

(180, <tf.Tensor: id=350264, shape=(), dtype=float32, numpy=0.9921875>)
(190, <tf.Tensor: id=351965, shape=(), dtype=float32, numpy=1.0>)
(200, <tf.Tensor: id=353666, shape=(), dtype=float32, numpy=1.0>)
(210, <tf.Tensor: id=355367, shape=(), dtype=float32, numpy=0.9921875>)
(220, <tf.Tensor: id=357068, shape=(), dtype=float32, numpy=0.984375>)
(230, <tf.Tensor: id=358769, shape=(), dtype=float32, numpy=0.9921875>)
(240, <tf.Tensor: id=360470, shape=(), dtype=float32, numpy=1.0>)
(250, <tf.Tensor: id=362171, shape=(), dtype=float32, numpy=1.0>)
(260, <tf.Tensor: id=363872, shape=(), dtype=float32, numpy=0.9921875>)
(270, <tf.Tensor: id=365573, shape=(), dtype=float32, numpy=0.9921875>)
(280, <tf.Tensor: id=367274, shape=(), dtype=float32, numpy=0.9921875>)
(290, <tf.Tensor: id=368975, shape=(), dtype=float32, numpy=1.0>)
(300, <tf.Tensor: id=370676, shape=(), dtype=float32, numpy=0.9921875>)
(310, <tf.Tensor: id=372377, shape=(), dtype=float32, numpy=0.9765625>)
(320, <tf.Tensor: id=37

In [11]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nTest Model \t\t Loss: %.6f\tAccuracy: %.6f' % (test_loss, test_acc))

10000/10000 [==============================] - 5s 499us/step

Test Model 		 Loss: 0.031740	Accuracy: 0.989500
